# A getting started tutorial of ModelCI installation, Register and convert Model

This is a getting started tutorial for those who are new to ML-ModelCI, by the end of this tutorial, you will be able to: 

- Setting up python environment required by ML-ModelCI.
- Start and stop ModelCI service.
- Master basic usages of ML-ModelCI, such as model loading, registering,retrieving and converting.
- Have a basic understanding of machine learning model lifecycle.

## 1. Installation

Here are some prequisities before installation

- Python version: 3.7
- Docker service installed and started
- Manually install [TensorRT](https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html) if your linux distribution is not Ubuntu

Firstly, we should install dependencies specified in `requirements.txt`

In [1]:
!pip install -q -r requirements.txt

Then we can install the ModelCI python package based on <https://github.com/cap-ntu/ML-Model-CI#using-pip>.

In [2]:
!pip install -q git+https://github.com/cap-ntu/ML-Model-CI.git@master --use-feature=2020-resolver

## 2. Start the ModelCI Service

Firstly, we should set some environmemnt variables, especially mongodb related variables, just make sure the port you specified is not occupied.

In [3]:
# set environment variables 
%env MONGO_HOST=localhost
%env MONGO_PORT=27017
%env MONGO_USERNAME=modelci
%env MONGO_PASSWORD=modelci@2020
%env MONGO_DB=modelci
%env MONGO_AUTH_SOURCE=modelci
%env BACKEND_CORS_ORIGINS="http://localhost,http://localhost:3000,http://localhost:8080,https://localhost:3000,https://localhost:8080"
%env PROJECT_NAME=modelci
%env SECRET_KEY=2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19

env: MONGO_HOST=localhost
env: MONGO_PORT=27017
env: MONGO_USERNAME=modelci
env: MONGO_PASSWORD=modelci@2020
env: MONGO_DB=modelci
env: MONGO_AUTH_SOURCE=modelci
env: BACKEND_CORS_ORIGINS="http://localhost,http://localhost:3000,http://localhost:8080,https://localhost:3000,https://localhost:8080"
env: PROJECT_NAME=modelci
env: SECRET_KEY=2a6c03b9ca06cd8fc3cf506f0ba924cb735f15918d54758426fd7282366a5e19


Then start the modelci service by following command:

In [4]:
!modelci start

2020-12-02 10:32:46.805488: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-12-02 10:32:53,276 - ml-modelci Docker Container Manager - INFO - Container name=mongo-76821 stared
2020-12-02 10:32:54,671 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-6052 started.
2020-12-02 10:32:57,019 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-91064 started.
2020-12-02 10:32:58,066 - ml-modelci Docker Container Manager - INFO - gpu-metrics-exporter-52271 stared
2020-12-02 10:32:58,289 - modelci backend - INFO - Uvicorn server listening on 8000


## 2. Register ResNet50 Model


Firstly, we load pre-trained resnet50 model from torchvision, you can refer to <https://pytorch.org/docs/stable/torchvision/models.html> for more examples of pretrained models.

In [5]:
# load resnet50 model from torchvision
from torchvision import models
model = models.resnet50(pretrained=True)

Then we register this model into ModelHub

In [6]:
from modelci.hub.manager import register_model
from modelci.types.bo import Framework, IOShape, ModelVersion,Engine
from modelci.types.trtis_objects import ModelInputFormat
# set model input and output formats
inputs = [IOShape([-1, 3, 224, 224], dtype=float, name='INPUT__0', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [IOShape([-1, 1000], dtype=float, name='probs')]
# register the model
register_model(
    model,
    dataset='imagenet',
    acc=0.76,
    task='image classification',
    inputs=inputs,
    outputs=outputs,
    architecture='ResNet50',
    framework=Framework.PYTORCH,
    version=ModelVersion('2')
)

2020-12-02 10:33:14,883 - converter - INFO - Use cached model
2020-12-02 10:33:14,884 - converter - INFO - Use cached model


## 3. Retrieve Models
By default, Converter will automatically convert registered models into optimized formats,PyTorch model can be converted to TorchScipt and ONNX formats, so we can retrieve two models from ModelHub.

In [7]:
# search for model
from modelci.hub.manager import retrieve_model
retrieved_models = retrieve_model(
        architecture_name = 'ResNet50',
        framework = Framework.PYTORCH,
        version=ModelVersion('2')
)

In [8]:
retrieved_models

We can compare detatiled information of these two models

In [9]:
retrieved_models[0].__dict__

{'_id': '5fc6fcedf2ede8193ad7095e',
 'name': 'ResNet50',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.TORCHSCRIPT: 2>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7f442046d2d0>,
 'dataset': 'imagenet',
 'acc': 0.76,
 'task': 'image classification',
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7f442046d710>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7f442046d110>],
 'weight': <modelci.types.bo.model_objects.Weight at 0x7f442046d610>,
 'profile_result': None,
 'status': <Status.RUNNING: 2>,
 'creator': 'sherry',
 'create_time': datetime.datetime(2020, 12, 2, 10, 33, 13, 880000)}

In [10]:
retrieved_models[1].__dict__

{'_id': '5fc6fceff2ede8193ad70ae8',
 'name': 'ResNet50',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.ONNX: 3>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7f4423080210>,
 'dataset': 'imagenet',
 'acc': 0.76,
 'task': 'image classification',
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7f44fbfa24d0>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7f44e95e6490>],
 'weight': <modelci.types.bo.model_objects.Weight at 0x7f44e8d56710>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'sherry',
 'create_time': datetime.datetime(2020, 12, 2, 10, 33, 13, 880000)}

## 4. Convert Models
We can convert models mannually. 

You can refer to <https://github.com/cap-ntu/ML-Model-CI/blob/master/docs/tutorial/convert.md> for more details.

In the following example, we will convert ONNX model into TensorRT format.

In [11]:
from modelci.hub.converter import TRTConverter
from modelci.hub.utils import generate_path
from modelci.types.bo import IOShape

# set model input and output formats
inputs = [IOShape([-1, 3, 224, 224], dtype=float, name='INPUT__0', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [IOShape([-1, 1000], dtype=float, name='probs')]

In [12]:
# get ONNX model saved path
onnx_path = retrieved_models[1].saved_path
onnx_path

PosixPath('/home/sherry/.modelci/ResNet50/pytorch-onnx/2.onnx')

In [13]:
# set TensorRT format model save path
save_path = generate_path(
    model_name='ResNet50', framework=Framework.PYTORCH,engine=Engine.TRT,version=2
)
save_path

PosixPath('/home/sherry/.modelci/ResNet50/pytorch-trt/2')

In [14]:
from modelci.hub.converter import PyTorchConverter
TRTConverter.from_onnx(onnx_path, save_path, inputs=inputs, outputs=outputs)

Loading ONNX file from path /home/sherry/.modelci/ResNet50/pytorch-onnx/2.onnx...


True

## 5.Stop the ModelCI Service

In [15]:
!modelci stop

2020-12-02 10:36:56.968220: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-12-02 10:36:59,967 - ml-modelci Docker Container Manager - INFO - Container name=gpu-metrics-exporter-52271 stopped.
2020-12-02 10:37:01,029 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-91064 stopped.
2020-12-02 10:37:01,560 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-6052 stopped.
2020-12-02 10:37:02,325 - ml-modelci Docker Container Manager - INFO - Container name=mongo-76821 stopped.
2020-12-02 10:37:02,448 - modelci backend - INFO - The Uvicorn server with pid=28282 stopped.


Then you can remove all the stoppped docker containers by the following command:

In [16]:
!docker rm $(docker ps -a -q)

543bbb7d4cf4
f54b6218bdfe
0db5da484143
4002cbdf701c


## License

```raw
Copyright 2020 Nanyang Technological University, Singapore

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```